In [1]:
import yfinance
from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed

In [2]:
symbol = "spy"
start_date = "1994-01-01"
end_date = "2021-08-12"
data = yfinance.download(symbol, start=start_date, end=end_date)
file_name = f"{symbol}.csv"
data.to_csv(file_name)
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1994-01-03,46.59375,46.65625,46.40625,46.46875,27.971474,960900
1994-01-04,46.53125,46.65625,46.46875,46.65625,28.084343,164300
1994-01-05,46.71875,46.78125,46.53125,46.75000,28.140778,710900
1994-01-06,46.81250,46.84375,46.68750,46.75000,28.140778,201000
1994-01-07,46.84375,47.06250,46.71875,47.03125,28.310053,775500


In [3]:
class BuyAndHoldStrategy(strategy.BacktestingStrategy):

    def __init__(self, feed, instrument):
        super(BuyAndHoldStrategy, self).__init__(feed)
        self.instrument = instrument
        self.setUseAdjustedValues(True)
        self.position = None

    def onEnterOk(self, position):
        self.info(f"{position.getEntryOrder().getExecutionInfo()}")
    
    def onBars(self, bars):
        bar = bars[self.instrument]

        if self.position is None:
            close = bar.getAdjClose()
            broker = self.getBroker()
            cash = broker.getCash()*.98
            quantity = cash / close

            self.position = self.enterLong(self.instrument, quantity)

In [4]:
# # Load the bar feed from the CSV file
feed = yahoofeed.Feed()
feed.addBarsFromCSV(symbol, file_name)

strategy = BuyAndHoldStrategy(feed, symbol)



In [5]:
strategy.run()

1994-01-04 00:00:00 strategy [INFO] 1994-01-04 00:00:00 - Price: 28.009100242660736 - Amount: 35035 - Fee: 0


In [6]:
portfolio_value = strategy.getBroker().getEquity() + strategy.getBroker().getCash()
print(portfolio_value)

15585234.60322943
